<a href="https://colab.research.google.com/github/paritoshkc/TextAnalytics/blob/master/Review_Scrapping.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import lxml.html as lh
import pandas as pd



# Get movie name and year list from wikipedia 
url_wiki='https://en.wikipedia.org/wiki/List_of_Academy_Award-winning_films'
#Create a handle, page, to handle the contents of the website
page = requests.get(url_wiki).text
soup= BeautifulSoup(page, "lxml")

My_table = soup.find('table',{'class':'wikitable sortable'})
links = My_table.find_all('a')

# for future use
name=[]
year=[]


# used for constrcution of urls
url_front = 'https://www.rottentomatoes.com/m/'
url_back_critics='/reviews'
url_back_audi='/reviews?type=user'

url_critics=[]
url_audi=[]
url_mid=[]


i=0
for link in links:
  texts.append(link.text.split('\n'))
for text in texts:
  if text[0]=='[1]':
    continue
  if i%2==0:
    text_string=text[0].replace(' ','_').lower()
    name.append(text_string)
  else:
    year.append(text[0])
    if int(text[0])<2018:
      break
  i+=1

zippedList =  list(zip(name,year))

for n,y in zippedList:
  mid=url_mid.append(n+'_'+y)

for mid in url_mid:
  url_critics.append(url_front+mid+url_back_critics)
  url_audi.append(url_front+mid+url_back_audi)

zippedList =  list(zip(url_mid,url_critics,url_audi))

# for future use
dfObj = pd.DataFrame(zippedList, columns = ['Name' , 'url_critics','url_audi']) 




In [0]:

def get_user_reviews():
    
  for url in url_audi:
    if '2019' in url:
      print(url)
      response = requests.get(url)

      soup = BeautifulSoup(response.text, "html.parser")


      # audience reviews
      audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
      if len(audience_reviews)!=0 :
        for reviews in audience_reviews:
          print(reviews.text)
      

In [121]:
get_user_reviews()

https://www.rottentomatoes.com/m/parasite_2019/reviews?type=user
This was the worst movie I have seen in a long time. Do not waste your money on seeing it. When the movie was over my wife and I sat there almost in shock and we both said, "How could this have been given the Oscar for the Best Movie of the Year? Just pass on this one and go see something else; anything else!
（9.0）Interesting！电影《寄生虫》很有趣，富人家地下室那段开始变得非常精彩。金基泽一家和雯光一家都代表的是穷人家庭，甚至男主人曾经开的都是台湾古早味蛋糕店，（而且最后都失败了）但他们两家人还是展开斗争，最后一把刀酿成三家人的悲剧。浮在水面上的神石，小儿子抽象的画作，难闻的气味……这些线索推动或暗示故事发展，最后发生了一个略荒诞的故事。
A beautifully photographed and acted,  fearless and fantastical exploration of class tension, poverty, entitlement, human nature, gullibility, and primal urges.  A tour de force deserving of best picture.
This movie traumatized my father, then it traumatized me. Why me. Thank you based Korea.
It was shot well but apart from that basic and boring
Highly overrated but worth a look 👍
Masterpiece! No way this movie can be pulled off in the United S

In [10]:


url = 'https://www.rottentomatoes.com/m/the_invisible_man_2020/reviews?type=verified_audience'
response = requests.get(url)

soup = BeautifulSoup(response.text, "html.parser")


# audience reviews
audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
for reviews in audience_reviews:
 print(reviews.text)

I love when watching a movie I cant figure out whats going to happen.
We really like the  movie is a lot of suspense and  was good ..
Suspenseful and engaging!  Nice work!
Excellent. Looking forward to part 2?
It was fresh and original.  The movie has a good twist to it.
Awesome movie loved it
A genius retelling on a classic. No cheap jump scares, but genuine suspense. Elizabeth Miss gives a brilliant performance and you feel her sanity being chipped away.
It started very slow. When it did pick up it turned out to be very good. The ending was great.
It was ok. Not the worst and not the best. It was a different take on the invisible man. The way he became invisible was a little weird.
This is the combined rating from two people. Loved the movie, but wish they showed more from what the lead character suffered to get to the point to leave her home in the first place.


In [114]:
audience_reviews=soup.findAll('p', attrs={'class': 'audience-reviews__review--mobile js-review-text clamp clamp-4 js-clamp'})
print('User Reviews')
for reviews in audience_reviews:
 print(reviews.text)


# audience_review1s=soup1.findAll('div', attrs={'class': 'the_review'})
# audience_review1s
# for reviews in audience_review1s:
#  print(reviews.text)

User Reviews


In [117]:

response = requests.get('https://www.rottentomatoes.com/m/la_la_land/reviews')
soup = BeautifulSoup(response.text, "html.parser")


import re
review_pat = re.compile(r'<div class=\"the_review\">[;a-zA-Z\s,-.\'\/\?\[\]\":\']*</div>')

soup = str(soup)

review_soup = soup.split('="review_table')[1].split('row review_table_row')
review_soup.pop(0)
review_soup = soup.split('="review_table')[1].split('row review_table_row')
review_soup.pop(0)
for review in review_soup:
        
        # extract review
        match = re.findall(review_pat, str(review))
        if len(match) > 0:
            m = match[0]
            for iden in ['<div class="the_review"> ','</div>']:
                m = m.replace(iden,'')
            reviews.append(m.strip('"'))
    


for review in reviews:
  if review.startswith('<div class="the_review">'):
    print(review.replace('<div class="the_review">','').replace('                                   ',''))
# for review in reviews:
#   print(review.split('\n')[1])
#   print('\n')


 Do yourself a favor and catch La La Land. It's a necessary reminder that romance, triumph, and hope don't just exist in the movies.
                                

 It'll have your heart aching with joy and yearning; its tunes will linger in your brain and hopefully weave a little bit of its magic into your life. Falling in love with it is inevitable.
                                

 An up-beat, enchanting, classic musical...
                                

 It's the careful balance between the emotional realism of the acting and the sheer technical audacity of the musical numbers that elevates the movie into the realm of greatness even before Chazelle rips out your heart with the extended, bittersweet coda.
                                

 [Stone and Gosling] might not have the fancy footwork of Fred Astaire and Ginger Rogers, but they're radiant together nonetheless, zinging through their rapport as well as their steps.
                                

 La La Land triumphs

https://www.rottentomatoes.com/m/parasite_2019/reviews?type=user
